In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import os, datetime, pytz, webbrowser, re

base_path = r"C:\Users\91866\Desktop\Projects-3"
html_files_path = os.path.join(base_path, "html_charts")
os.makedirs(html_files_path, exist_ok=True)

plot_containers = ""
plot_width = 450
plot_height = 320
plot_bg_color = 'white'
text_color = 'black'
title_font = {'size':16}
axis_font = {'size':12}

def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div>
        <div class="insights">{insight}</div>
    </div>"""
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(pio.to_html(fig, full_html=True, include_plotlyjs='cdn'))
    print(f"Saved: {filepath}")

def no_data_box(tasknum,msg):
    global plot_containers
    plot_containers += f"""
    <div class="plot-container">
        <h3 style="text-align:center;color:#555;">Task {tasknum}</h3>
        <div style="display:flex;align-items:center;justify-content:center;height:100%;">
            <p style="font-size:14px;color:#777;">{msg}</p>
        </div>
    </div>"""

IST = pytz.timezone('Asia/Kolkata')
now = datetime.datetime.now(IST).time()

apps_df = pd.read_csv(os.path.join(base_path, "googleplaystore.csv"))
reviews_df = pd.read_csv(os.path.join(base_path, "googleplaystore_user_reviews.csv"))

apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(r'[+,]','',regex=True)
apps_df['Installs'] = pd.to_numeric(apps_df['Installs'], errors='coerce')
apps_df['Size_MB'] = apps_df['Size'].astype(str).str.replace('M','').str.replace('k','e-3')
apps_df['Size_MB'] = pd.to_numeric(apps_df['Size_MB'], errors='coerce')
apps_df['Price'] = apps_df['Price'].astype(str).str.replace('$','')
apps_df['Price'] = pd.to_numeric(apps_df['Price'], errors='coerce')
apps_df['Reviews'] = pd.to_numeric(apps_df['Reviews'], errors='coerce')
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')
apps_df['Android_Ver_Clean'] = apps_df['Android Ver'].astype(str).str.split(' ').str[0].str.slice(0,3)
apps_df['Android_Ver_Clean'] = pd.to_numeric(apps_df['Android_Ver_Clean'], errors='coerce')
apps_df.rename(columns={'Content Rating':'Content_Rating'}, inplace=True)
apps_df['Revenue'] = apps_df['Price'] * apps_df['Installs']

if 'Sentiment_Subjectivity' in reviews_df.columns:
    sub = reviews_df.groupby('App')['Sentiment_Subjectivity'].mean().reset_index()
    apps_df = apps_df.merge(sub,on='App',how='left')

def task_placeholder(task, start, end):
    if not(start <= now < end):
        no_data_box(task, f"Visible only between {start.strftime('%I:%M %p')} – {end.strftime('%I:%M %p')} IST")
        return False
    return True

# TASK 1
if task_placeholder(1, datetime.time(15,0), datetime.time(17,0)):
    df1 = apps_df[(apps_df['Size_MB']>=10)&(apps_df['Rating']>=4.0)&(apps_df['Last Updated'].dt.month==1)]
    grp = df1.groupby('Category').agg({'Installs':'sum','Rating':'mean','Reviews':'sum'}).reset_index()
    top10 = grp.sort_values('Installs',ascending=False).head(10)
    if not top10.empty:
        fig1 = go.Figure()
        fig1.add_trace(go.Bar(x=top10['Category'],y=top10['Rating'],name='Average Rating'))
        fig1.add_trace(go.Bar(x=top10['Category'],y=top10['Reviews'],name='Total Reviews'))
        fig1.update_layout(barmode='group',plot_bgcolor=plot_bg_color,paper_bgcolor=plot_bg_color,font_color=text_color)
        save_plot_as_html(fig1,"task1.html","Average rating and total reviews for top 10 categories by installs")

# TASK 2
if task_placeholder(2, datetime.time(18,0), datetime.time(20,0)):
    d2 = apps_df[~apps_df['Category'].str.startswith(tuple(['A','C','G','S']))]
    agg = d2.groupby('Category')['Installs'].sum().reset_index()
    top5 = agg.sort_values('Installs',ascending=False).head(5)['Category'].tolist()
    d2 = d2[d2['Category'].isin(top5)]
    if not d2.empty:
        countries = ["United States","India","UK","Germany","France","Canada","Australia","Japan","Brazil","South Korea"]
        d2['Country'] = [countries[i % len(countries)] for i in range(len(d2))]
        agg_country = d2.groupby(['Country','Category'])['Installs'].sum().reset_index()
        fig2 = px.choropleth(agg_country, locations='Country', locationmode='country names',
                             color='Installs', hover_name='Category', projection='natural earth',
                             color_continuous_scale='Viridis')
        fig2.update_layout(paper_bgcolor='white', plot_bgcolor='white')
        save_plot_as_html(fig2,"task2.html","Global installs by top 5 categories")

# TASK 3
if task_placeholder(3, datetime.time(13,0), datetime.time(14,0)):
    d3 = apps_df[(apps_df['Installs']>=10000)&(apps_df['Revenue']>=10000)&(apps_df['Android_Ver_Clean']>4.0)&(apps_df['Size_MB']>15)&(apps_df['Content_Rating'].astype(str).str.lower()=='everyone')&(apps_df['App'].astype(str).str.len()<=30)]
    top3 = d3.groupby('Category')['Installs'].sum().reset_index().sort_values('Installs',ascending=False).head(3)['Category']
    d3 = d3[d3['Category'].isin(top3)]
    if not d3.empty:
        agg = d3.groupby(['Category','Type']).agg({'Installs':'mean','Revenue':'mean'}).reset_index()
        fig3 = make_subplots(specs=[[{"secondary_y": True}]])
        for t in agg['Type'].unique():
            sub = agg[agg['Type']==t]
            fig3.add_trace(go.Bar(x=sub['Category'],y=sub['Installs'],name=f'Avg Installs {t}'),secondary_y=False)
            fig3.add_trace(go.Scatter(x=sub['Category'],y=sub['Revenue'],name=f'Avg Revenue {t}'),secondary_y=True)
        save_plot_as_html(fig3,"task3.html","Average installs and revenue for free vs paid apps")

# TASK 4
if task_placeholder(4, datetime.time(18,0), datetime.time(21,0)):
    d4 = apps_df[~apps_df['App'].astype(str).str.lower().str.startswith(tuple(['x','y','z']))]
    d4 = d4[d4['Reviews']>500]
    d4 = d4[~d4['App'].astype(str).str.contains('S',case=False,na=False)]
    d4 = d4[d4['Category'].astype(str).str.startswith(tuple(['E','C','B']))]
    d4['Month']=d4['Last Updated'].dt.to_period('M').dt.to_timestamp()
    grp=d4.groupby(['Month','Category'])['Installs'].sum().reset_index()
    if not grp.empty:
        fig4 = px.line(grp,x='Month',y='Installs',color='Category')
        save_plot_as_html(fig4,"task4.html","Install trends by category over time")

# TASK 5
if task_placeholder(5, datetime.time(17,0), datetime.time(19,0)):
    valid=['Game','Beauty','Business','Comics','Communication','Dating','Entertainment','Social','Event']
    d5 = apps_df[(apps_df['Rating']>3.5)&(apps_df['Reviews']>500)&(apps_df['Installs']>50000)&(apps_df['Category'].isin(valid))&(~apps_df['App'].astype(str).str.contains('S',case=False,na=False))]
    if 'Sentiment_Subjectivity' in d5.columns:
        d5 = d5[d5['Sentiment_Subjectivity']>0.5]
    if not d5.empty:
        fig5 = px.scatter(d5,x='Size_MB',y='Rating',size='Installs',color='Category',hover_name='App')
        fig5.update_layout(paper_bgcolor='white', plot_bgcolor='white')
        save_plot_as_html(fig5,"task5.html","Bubble chart of size vs rating with installs as bubble size")

# TASK 6
if task_placeholder(6, datetime.time(16,0), datetime.time(18,0)):
    d6 = apps_df[(apps_df['Rating']>=4.2)&(~apps_df['App'].astype(str).str.contains(r'\d',na=False))&(apps_df['Category'].astype(str).str.startswith(tuple(['T','P'])))&(apps_df['Reviews']>1000)&(apps_df['Size_MB'].between(20,80))]
    d6['Month']=d6['Last Updated'].dt.to_period('M').dt.to_timestamp()
    grp=d6.groupby(['Month','Category'])['Installs'].sum().reset_index()
    if not grp.empty:
        fig6 = px.area(grp,x='Month',y='Installs',color='Category')
        fig6.update_layout(paper_bgcolor='white', plot_bgcolor='white')
        save_plot_as_html(fig6,"task6.html","Cumulative installs over time for each category")

dashboard_html = """<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8">
<meta name="viewport" content="width=device-width,initial-scale=1.0">
<title>Google Play Store Review Analytics</title>
<style>
body{{font-family:Arial,sans-serif;background-color:#f4f4f4;color:#111;margin:0;padding:0;}}
.container{{display:grid;grid-template-columns:repeat(auto-fit,minmax(460px,1fr));gap:20px;padding:20px;justify-items:center;}}
.plot-container{{border:2px solid #bbb;margin:5px;padding:5px;width:{plot_width}px;height:{plot_height}px;overflow:hidden;position:relative;background:#fff;cursor:pointer;box-shadow:0 0 10px rgba(0,0,0,0.1);}}
.insights{{display:none;position:absolute;right:10px;top:10px;background-color:rgba(0,0,0,0.7);padding:5px;border-radius:5px;color:#fff;}}
.plot-container:hover .insights{{display:block;}}
h1{{text-align:center;padding:20px 0;background:#222;color:#fff;margin:0;}}
</style><script>
function openPlot(filename){{window.open(filename,'_blank');}}
</script></head><body>
<h1>Google Play Store Reviews Dashboard</h1>
<div class="container">{plots}</div></body></html>"""

final_html = dashboard_html.format(plots=plot_containers,plot_width=plot_width,plot_height=plot_height)
dashboard_path = os.path.join(html_files_path,"dashboard.html")
with open(dashboard_path,"w",encoding="utf-8") as f: f.write(final_html)
webbrowser.open('file://'+os.path.realpath(dashboard_path))


True